# Floyd-Warshall algorithm

The [Floyd-Warshall algorithm](https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm) calculates the shortest distance between all node pairs in a graph.

The algorithm works for directed, weighted graphs.

The compute complexity for the algorithm is $O(n^3)$.


In [1]:
# Source: ChatGPT, with minor edits. Notably I added the docstring, and the 
# algorithm ChatGPT provided only worked for graphs with keys that were integers.
def floyd_warshall(graph):
    """Find the shortest path between all node pairs in a graph using the 
    Floyd-Warshall algorithm.

    Parameters
    ----------
    graph : dict of dicts
        Each key in outer dict is a node, and each value is another dict. Within
        the inner dict, each key is an adjacent/neighbor node to the outer dict
        key, and each value is the edge weight.

    Returns
    -------
    dist : list of lists
        An n x n matrix, with each cell containing the shortest distance between
        any two nodes. All the diagonals will be 0s and the upper right and lower
        left triangles will be identical.
    """
    # Initialize distance matrix with values of infinity in all the cells
    nodes = list(graph.keys())
    num_nodes = len(nodes)
    dist = [[float('inf')] * num_nodes for _ in range(num_nodes)]

    # Then replace those values with 0s for the diagonals and with edge weight
    # values where possible.
    for i, node in enumerate(nodes):
        dist[i][i] = 0
        for neighbor, edge_weight in graph[node].items():
            dist[i][nodes.index(neighbor)] = edge_weight

    # Finally, update the distance matrix using Floyd-Warshall algorithm, which
    # ...
    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])

    return dist


Set up an example graph to test algorithm on:

In [2]:
graph = {
    'a': {1: 3, 2: 6},
    1: {'a': 3, 2: 2},
    2: {'a': 6, 1: 2}
}
graph

{'a': {1: 3, 2: 6}, 1: {'a': 3, 2: 2}, 2: {'a': 6, 1: 2}}

In [3]:
floyd_warshall(graph)

[[0, 3, 5], [3, 0, 2], [5, 2, 0]]

In [5]:
from collections import defaultdict
from itertools import product

def fw(graph):
    """Find the shortest path between all node pairs in a graph using the 
    Floyd-Warshall algorithm.

    Parameters
    ----------
    graph : dict of dicts
        Each key in outer dict is a node, and each value is another dict. Within
        the inner dict, each key is an adjacent/neighbor node to the outer dict
        key, and each value is the edge weight.

    Returns
    -------
    dist : dict
        Keys are (i, j) tuples, for all pairs of nodes i and j, and values are
        the shortest weighted distance between node i and node j in the graph.
    """
    # Create an empty default dictionary, with default values set at infinity.
    # Note that instead of an n x n matrix being returned, as is typical for 
    # most implementations of Floyd-Warshall, this will return a dictionary,
    # where the keys are an (i, j) tuple and the values are the min distance
    # between nodes i and j.
    dist = defaultdict(lambda: float("inf"))

    # Initialize the dist[(i, j)] values, where possible:
    # Diagonals will be set to 0s and direct neighbors will be given their
    # edge weights.
    for node, neighbors in graph.items():
        dist[(node, node)] = 0
        for neighbor, edge_weight in neighbors.items():
            dist[(node, neighbor)] = edge_weight

    # Update initialized distances
    # (use itertools.product for a aore efficient way to do for k: for i: for j:)
    for k, i, j in product(graph.keys(), repeat=3):
        dist[(i, j)] = min(dist[(i, j)], dist[(i, k)] + dist[(k, j)])

    return dict(dist)

In [6]:
fw(graph)

{('a', 'a'): 0,
 ('a', 1): 3,
 ('a', 2): 5,
 (1, 1): 0,
 (1, 'a'): 3,
 (1, 2): 2,
 (2, 2): 0,
 (2, 'a'): 5,
 (2, 1): 2}

Let's compare the runtimes for both implementations:

In [7]:
%timeit floyd_warshall(graph)
%timeit fw(graph)

28.9 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
31 µs ± 505 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Hmm... Interestingly, the first implementation is marginally faster...

I wonder if the test graph is too small to make the overhead of `collections.defaultdict()` and `itertools.product()` worth it. 

Perhaps with larger graphs it would be? 